# Steps in Principal Component Analysis
Source: https://online.stat.psu.edu/stat505/lesson/11 lessons 11 and 12.

You can check our results against theirs starting at 11.5.

## 0: What does Principal Component Analysis (PCA) do?
The goal of PCA is to reduce the dimensionality of input data.
Given a large number of input variables, PCA allows us to explain most of the variability of the data with far fewer variables.
Principal components are chosen in a way that explains as much of the variation within the data as possible.
The first component explains the most variation, followed by the second, etc.
Visually, principal components are lines through the data that are as close to the cloud of data as possible, while remaining perpendicular to each other.

This YouTube video does a good job of explaining generally what PCA does without explaining any of the math: https://youtu.be/pmG4K79DUoI.
The same guy has a follow up video to explain the math, if anyone is interested: https://youtu.be/dhK8nbtii6I.

In the rest of this script, we will see step-by-step the operations needed to conduct PCA.

In [1]:
# Import packages
import pandas as pd
import numpy as np

# Import data
places = pd.read_csv("places.csv")

# Drop id column
places = places.drop(axis = 1, columns = {"id"})

## 1: Standardization
Calculate the [Z-score](https://www.investopedia.com/terms/z/zscore.asp) for every variable.
This prevents any one variable from having more influence than any other variable just because its values and variance tend to be larger.


In [2]:
# Scale data
for col in places.columns:
    places[col] = np.log10(places[col]) # the tutorial took a log transform to deal with skew
    places[col] = ( places[col] - np.mean(places[col]) ) / np.std(places[col], ddof=1) # ddof = 1 is to compute sample standard deviation instead of population standard deviation
    # In some instances, it is preferred to center rather than scale your data. To center, comment out the line above and uncomment the line below 
    # places[col] = ( places[col] - np.mean(places[col]) ) 

## 2: Covariance matrix
Calculate the [covariance](https://corporatefinanceinstitute.com/resources/data-science/covariance/) between each variable in the standardized dataset and construct a covariance matrix. 
If we call the standardized matrix $X$, then we calculate the covariance matrix as $X^T X \frac{1}{n-1}$.
Covariance measures the level of variability between two variables.
Note that the covariance matrix of the standardized data is equivalent to the Pearson correlation matrix of the original data.

In [3]:
# Calculate covariance matrix (covariance matrix of standardized variables is the same as a correlation matrix of the original variables)
cov_matrix = (places.T).dot(places)*(1/ (len(places)-1))
cov_matrix

,climate,housing,health,crime,trans,educate,arts,recreate,econ
climate,1.000000,0.272964,0.150561,0.227751,0.021559,0.077458,0.172683,0.120610,-0.100727
housing,0.272964,1.000000,0.431935,0.139234,0.317732,0.202088,0.508501,0.460696,0.297058
health,0.150561,0.431935,1.000000,0.183625,0.418850,0.464764,0.678129,0.254036,0.054047
crime,0.227751,0.139234,0.183625,1.000000,0.273852,0.055508,0.346462,0.292124,0.276182
trans,0.021559,0.317732,0.418850,0.273852,1.000000,0.311237,0.547634,0.390684,0.062680
educate,0.077458,0.202088,0.464764,0.055508,0.311237,1.000000,0.347899,0.093001,0.128858
arts,0.172683,0.508501,0.678129,0.346462,0.547634,0.347899,1.000000,0.496519,0.134761
recreate,0.120610,0.460696,0.254036,0.292124,0.390684,0.093001,0.496519,1.000000,0.175914
econ,-0.100727,0.297058,0.054047,0.276182,0.062680,0.128858,0.134761,0.175914,1.000000


## 3: Eigenvalues and eigenvectors
Compute [eigenvalues and eigenvectors](https://math.libretexts.org/Bookshelves/Linear_Algebra/A_First_Course_in_Linear_Algebra_(Kuttler)/07%3A_Spectral_Theory/7.01%3A_Eigenvalues_and_Eigenvectors_of_a_Matrix) of the covariance matrix.
An eigenvector $x$ of a matrix $A$ is one such that $Ax = \lambda x$ for some number $\lambda$ (left-multiplying the vector by the matrix is the same as multiplying the vector by some constant $\lambda$).
The eigenvectors are our principal components.
Each component has an entry for every variable; these entries are called weights. 

In [4]:
# Convert to NumPy array
cov_matrix = cov_matrix.values

# Calculate eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

## 4: Rank components
Rank the eigenvectors from most to least important by their eigenvalues.
The bigger the eigenvalue, the more variance explained by its corresponding component, the more important the component.

In [5]:
# Combine eigenvalues and eigenvectors into one list
eigenzip = list(zip(eigenvalues, eigenvectors.T))

# Sort by eigenvalue 
sorted_eigenzip = sorted(eigenzip, key = lambda x:x[0], reverse = True)

## 5: Calculate percent variance explained 
This formula calculates the proportion of variance explained by the ith of all n eigenvectors:

$$ \frac{\lambda_i}{\lambda_1 + \lambda_2 + ... + \lambda_n} $$

And this formula calculates the proportion of variance explained by the first r of all n eigenvectors.

$$ \frac{\lambda_1 + \lambda_2 + ... + \lambda_r}{\lambda_1 + \lambda_2 + ... + \lambda_n} $$


In [6]:
tot = eigenvalues.sum()

eigenval_summary = pd.DataFrame( {"Component": np.NaN,
                                  "Eigenvalue": np.NaN,
                                  "Proportion": np.NaN,
                                  "Cumulative": np.NaN},
                               index = [i for i in range(len(eigenvalues))])

eigenvecs_sorted = pd.DataFrame(eigenvectors)

cumulative = 0

for i in range(len(sorted_eigenzip)):
    eigenval_summary["Component"][i] = i+1
    
    eigenval_summary["Eigenvalue"][i] = sorted_eigenzip[i][0]
    
    eigenval_summary["Proportion"][i] = eigenval_summary["Eigenvalue"][i]/tot
    
    cumulative += eigenval_summary["Proportion"][i]
    eigenval_summary["Cumulative"][i] = cumulative
    
    eigenvecs_sorted[i] = sorted_eigenzip[i][1]

In [7]:
display(eigenval_summary)
print("Total eigenvalue sum:", tot)

,Component,Eigenvalue,Proportion,Cumulative
0,1.0,3.297793,0.366421,0.366421
1,2.0,1.213562,0.134840,0.501262
2,3.0,1.105530,0.122837,0.624098
3,4.0,0.907280,0.100809,0.724907
4,5.0,0.860629,0.095625,0.820533
5,6.0,0.562186,0.062465,0.882998
6,7.0,0.483821,0.053758,0.936756
7,8.0,0.318072,0.035341,0.972097
8,9.0,0.251128,0.027903,1.000000


Total eigenvalue sum: 9.0


## 6: Calculate loadings

Apparently there is a difference between eigenvectors and the vectors of loadings.
By multiplying by the square root of the eigenvalue, our results become more interpretable, as we shall see below.
First we will inspect the original eigenvectors, then we will calculate loadings and inspect them again.

In [8]:
# Reverse signs
eigenvecs_sorted = (eigenvecs_sorted*-1)#.round(3) # for some reason everything had the opposite sign in ours...
# The choice of sign for eigenvectors is arbitrary... but how does that affect our SoVI calculation? 
# https://github.com/tidymodels/recipes/issues/653
# https://www.mathworks.com/help/stats/pca.html

# Inspect components
eigenvecs_sorted = eigenvecs_sorted.set_index(keys = pd.Index(["climate", "housing", "health", "crime", "trans", "educate", "arts", "recreate", "economy"]))
eigenvecs_sorted

,0,1,2,3,4,5,6,7,8
climate,0.157941,0.068629,0.799710,-0.376810,0.041046,0.216695,-0.151352,0.341128,-0.030098
housing,0.384405,0.139209,0.079616,-0.196543,-0.579868,-0.082220,-0.275197,-0.606101,0.042269
health,0.409910,-0.371812,-0.019475,-0.112522,0.029569,-0.534876,0.134975,0.150058,-0.594128
crime,0.259102,0.474132,0.128467,0.042300,0.692171,-0.139901,0.109504,-0.420125,-0.051012
trans,0.374889,-0.141486,-0.141068,0.430077,0.191416,0.323891,-0.678567,0.118833,-0.135843
educate,0.274325,-0.452355,-0.241056,-0.456943,0.224744,0.526583,0.262096,-0.211175,0.110124
arts,0.473847,-0.104410,0.011026,0.146881,0.011930,-0.321057,0.120499,0.259867,0.746727
recreate,0.353412,0.291942,0.041816,0.404019,-0.305654,0.394139,0.553094,0.137718,-0.226365
economy,0.164013,0.540453,-0.507310,-0.475780,-0.037108,-0.000974,-0.146867,0.414774,-0.047903


In [9]:
# Just illustrating that the eigenvectors are of unit length
(eigenvecs_sorted**2).sum(0) 

0    1.0
1    1.0
2    1.0
3    1.0
4    1.0
5    1.0
6    1.0
7    1.0
8    1.0
dtype: float64

In [10]:
# Calculate factor loadings
# Note: may need a different procedure if you chose to center rather than standardize the data
factor_loadings = eigenvecs_sorted
for i in range( len(eigenvecs_sorted) ):
    factor_loadings[i] = factor_loadings[i]*np.sqrt(eigenval_summary["Eigenvalue"][i])
factor_loadings#.round(3)

,0,1,2,3,4,5,6,7,8
climate,0.286819,0.075603,0.840848,-0.358916,0.038078,0.162476,-0.105276,0.192389,-0.015083
housing,0.698073,0.153355,0.083712,-0.187210,-0.537944,-0.061648,-0.191419,-0.341828,0.021182
health,0.744389,-0.409595,-0.020477,-0.107179,0.027432,-0.401045,0.093885,0.084629,-0.297733
crime,0.470524,0.522313,0.135076,0.040291,0.642128,-0.104896,0.076168,-0.236942,-0.025563
trans,0.680792,-0.155864,-0.148325,0.409653,0.177577,0.242851,-0.471992,0.067019,-0.068075
educate,0.498170,-0.498323,-0.253456,-0.435244,0.208495,0.394827,0.182307,-0.119098,0.055186
arts,0.860498,-0.115020,0.011593,0.139906,0.011068,-0.240726,0.083815,0.146560,0.374205
recreate,0.641790,0.321609,0.043967,0.384833,-0.283555,0.295522,0.384717,0.077670,-0.113438
economy,0.297846,0.595373,-0.533407,-0.453186,-0.034425,-0.000730,-0.102157,0.233924,-0.024005


Note: we can think of factor loadings as correlations between factors and input variables.

In [11]:
# Sum of squared loadings: relationship with eigenvalue
(factor_loadings**2).sum(0)

0    3.297793
1    1.213562
2    1.105530
3    0.907280
4    0.860629
5    0.562186
6    0.483821
7    0.318072
8    0.251128
dtype: float64

Column-wise, we can see that the sum of the squared loadings for a component equals the component's eigenvalue.

In [12]:
# Communalities
(factor_loadings**2).sum(1)

climate     1.0
housing     1.0
health      1.0
crime       1.0
trans       1.0
educate     1.0
arts        1.0
recreate    1.0
economy     1.0
dtype: float64

Row-wise, we can see that the 9 components explain 100% of variance  for each variable, because for each variable, the sum of the squared loadings amongst all of the components is 1.
For a given variable, the squared corresponding entry in a component tells you how much variance of that variable is explained by that component.

## 7: Select components

Initially, we derive the same number of components as there are variables in the model.
Since the goal is to reduce the dimensionality of the dataset, we typically seek to select a subset of the components.
In the Penn State tutorial, the authors select the first three components, and in order to check our results against theirs, we do the same here.
SoVI employs the Kaiser criterion, which means retaining all components with eigenvalues of at least 1

In [13]:
top3 = factor_loadings[[0,1,2]]

## 8: Calculate communalities

Communalities are calculated as
$\sum_{j=1}^{m} l_{ij}^2$,
where $m$ is the number of retained components and $l_{ij}$ is the loading corresponding to variable $i$ and component $j$. 

In [14]:
# Communalities of first three components
communalities = pd.DataFrame(columns = {"Variable", "Communality"}, index = [i for i in range(len(factor_loadings))])
for i in range(len(top3)):
    communalities["Variable"][i] = top3.index[i]
    communalities["Communality"][i] = (top3.iloc[i]**2).sum()
communalities

,Variable,Communality
0,climate,0.795007
1,housing,0.517832
2,health,0.722302
3,crime,0.512449
4,trans,0.509772
5,educate,0.560739
6,arts,0.753821
7,recreate,0.517259
8,economy,0.727704


Communalities can be interpreted like $R^2$ values in regression models. For example, the first three factors explain 79.5% of the variation in the climate variable.

In [15]:
# Total communality value
communalities["Communality"].sum()

5.616884741577941

In [16]:
# Proportion of total variance explained by the 3 factors
communalities["Communality"].sum()/len(factor_loadings)

0.6240983046197712

In [17]:
# Note, we get the same number by calculating the proportion of variance explained using eigenvalues
eigenval_summary["Eigenvalue"][0:3].sum()/eigenval_summary["Eigenvalue"].sum()

0.6240983046197713

## 9: Calculating scores
To calculate component scores, we first have to scale the components by dividing them by their eigenvalue.
Calculating scores is equivalent to translating them to a new coordinate system, where the axes are the principal components.

In [18]:
# Compute component weight matrix
# see the "computation of B in PCA" in the following link
# https://stats.stackexchange.com/questions/126885/methods-to-compute-factor-scores-and-what-is-the-score-coefficient-matrix-in
component_weight_matrix = factor_loadings/eigenvalues
component_weight_matrix

,0,1,2,3,4,5,6,7,8
climate,0.086973,0.062299,0.760584,-1.429213,0.119716,0.335818,-0.187262,0.212050,-0.017525
housing,0.211679,0.126368,0.075721,-0.745475,-1.691264,-0.127419,-0.340491,-0.376762,0.024612
health,0.225723,-0.337515,-0.018523,-0.426789,0.086243,-0.828912,0.167000,0.093278,-0.345949
crime,0.142679,0.430396,0.122182,0.160440,2.018811,-0.216808,0.135485,-0.261156,-0.029703
trans,0.206439,-0.128435,-0.134167,1.631252,0.558291,0.501944,-0.839566,0.073868,-0.079099
educate,0.151062,-0.410628,-0.229262,-1.733154,0.655496,0.816060,0.324282,-0.131270,0.064123
arts,0.260931,-0.094779,0.010487,0.557111,0.034796,-0.497551,0.149089,0.161537,0.434804
recreate,0.194612,0.265012,0.039771,1.532417,-0.891481,0.610808,0.684323,0.085608,-0.131808
economy,0.090317,0.490599,-0.482490,-1.804602,-0.108230,-0.001509,-0.181713,0.257830,-0.027893


In [19]:
# Compute scores
scores = pd.DataFrame(places.dot(component_weight_matrix[[0,1,2]].values))
scores

,0,1,2
0,-0.660169,1.338368,-0.898126
1,0.516697,-0.225532,1.051808
2,-1.292685,0.308392,0.024139
3,0.759513,-1.464882,-1.193371
4,1.342175,0.173920,0.394625
...,...,...,...
324,0.081002,-1.691177,-0.042397
325,-0.648080,1.002819,0.565105
326,-1.007456,-1.208724,0.179566
327,-0.196986,-1.209961,0.993437


## 10: Varimax rotation

Sometimes, after conducting Principal Component Analysis, researchers employ "rotations" to make their components more interpretable.
There are a number of different methodologies for rotating components, but they fall into two main classes: orthogonal rotations and oblique rotations. 

With an orthogonal rotation, the idea is to plot the factor loadings on the axes of our chosen components and then rotate those axes while preserving orthogonality (all components are perpendicular to each other) until the points are as close to the axes as possible.
The goal is to have each factor loads highly on a smaller number of variables.
The varimax rotation, used when calculating SoVI, is an example of an orthogonal rotation.
This [YouTube video](https://youtu.be/AjrU9oV3MRM) helps visualize what varimax rotation looks like.

With an oblique rotation, factors are allowed to become correlated with each other and they are no longer all orthogonal to each other.

In [20]:
%%script echo skipping # was trying to do this manually, no longer needed
# Calculate magnitude of row vectors
sums = np.sqrt((top3**2).sum(axis = 1))

# Normalize row vectors
scaled_loadings = top3.divide(sums, axis = 0)

# Check row magnitudes are 1
(scaled_loadings**2).sum(axis = 1)

skipping # was trying to do this manually, no longer needed


In [21]:
# Define a function to do Varimax rotation
# The following code is adapted from https://factor-analyzer.readthedocs.io/en/latest/_modules/factor_analyzer/rotator.html
# I simply copied it over because I was struggling to understand how varimax rotation works -- I'm struggling to find resources online that explain it in enough detail without being prohibitively mathematically dense
def varimax(loadings, normalize = True): 
    X = loadings.copy()
    n_rows, n_cols = X.shape
    if n_cols < 2:
        return X

    # normalize the loadings matrix
    # using sqrt of the sum of squares (Kaiser)
    if normalize == True:
        normalized_mtx = np.apply_along_axis(
            lambda x: np.sqrt(np.sum(x**2)), 1, X.copy()
        )
        X = (X.T / normalized_mtx).T

    # initialize the rotation matrix
    # to N x N identity matrix
    rotation_mtx = np.eye(n_cols)

    d = 0
    for _ in range(500):

        old_d = d

        # take inner product of loading matrix
        # and rotation matrix
        basis = np.dot(X, rotation_mtx)

        # transform data for singular value decomposition using updated formula :
        # B <- t(x) %*% (z^3 - z %*% diag(drop(rep(1, p) %*% z^2))/p)
        diagonal = np.diag(np.squeeze(np.repeat(1, n_rows).dot(basis**2)))
        transformed = X.T.dot(basis**3 - basis.dot(diagonal) / n_rows)

        # perform SVD on
        # the transformed matrix
        U, S, V = np.linalg.svd(transformed)

        # take inner product of U and V, and sum of S
        rotation_mtx = np.dot(U, V)
        d = np.sum(S)

        # check convergence
        if d < old_d * (1 + 1e-5):
            break

    # take inner product of loading matrix
    # and rotation matrix
    X = np.dot(X, rotation_mtx)

    # de-normalize the data
    if normalize == True:
        X = X.T * normalized_mtx
    else:
        X = X.T

    # convert loadings matrix to data frame
    loadings = X.T.copy()
    return loadings, rotation_mtx

In [22]:
# Implement varimax rotation
loadings, rotation_mtx = varimax(top3, normalize = True)

# View rotated loadings
rotated_loadings = pd.DataFrame(loadings, index = ["climate", "housing", "health", "crime", "trans", "educate", "arts", "recreate", "economy"])
rotated_loadings

,0,1,2
climate,0.021419,0.236416,0.859451
housing,0.438154,0.545767,0.167306
health,0.829227,0.126177,0.136981
crime,0.031006,0.701044,0.141509
trans,0.652352,0.288896,-0.027356
educate,0.733441,-0.094397,-0.117867
arts,0.738407,0.430850,0.151473
recreate,0.301406,0.645226,0.100488
economy,-0.022131,0.652516,-0.549032


## 11: Explained variance and communalities after rotation

In [23]:
variation_explained = pd.DataFrame({"Factor": [1,2,3],
                                    "Original": (top3**2).sum(axis = 0),
                                    "Rotated": (rotated_loadings**2).sum(axis = 0)})
print("Variation Explained:")
display(variation_explained)
print("Original total:", variation_explained["Original"].sum())
print("Rotated total:", variation_explained["Original"].sum())

Variation Explained:


,Factor,Original,Rotated
0,1,3.297793,2.481095
1,2,1.213562,1.981235
2,3,1.105530,1.154555


Original total: 5.616884741577941
Rotated total: 5.616884741577941


As you can see, the total amount of variation explained by the model did not change due to the rotation.
However, the amount of variation explained by the first factor dropped substantially, spreading its explained variance to the other two factors.
Thus rotations afford us with a cleaner interpretation of the data while explaining the same amount of variation in the data, although the amount of variation explained by each individual factor may change.

In [24]:
# Communalities of first three components
communalities_r = pd.DataFrame(columns = {"Variable", "Rotated Communality"}, index = [i for i in range(len(rotated_loadings))])
for i in range(len(top3)):
    communalities_r["Variable"][i] = rotated_loadings.index[i]
    communalities_r["Rotated Communality"][i] = (rotated_loadings.iloc[i]**2).sum()
communalities.merge(communalities_r, on = "Variable")

,Variable,Communality,Rotated Communality
0,climate,0.795007,0.795007
1,housing,0.517832,0.517832
2,health,0.722302,0.722302
3,crime,0.512449,0.512449
4,trans,0.509772,0.509772
5,educate,0.560739,0.560739
6,arts,0.753821,0.753821
7,recreate,0.517259,0.517259
8,economy,0.727704,0.727704


As you can see, the communalities do not change under rotation!

## 12: Calculating scores after rotation

In [25]:
# Compute component weight matrix
# see the "PCA's method" in the following link; have to do some matrix manipulation to get our exact implementation
# https://stats.stackexchange.com/questions/126885/methods-to-compute-factor-scores-and-what-is-the-score-coefficient-matrix-in
rotated_weight_matrix = np.dot(rotated_loadings, np.linalg.inv(np.dot(rotated_loadings.T, rotated_loadings)))
rotated_weight_matrix

array([[-0.11169719,  0.10434053,  0.75270875],
       [ 0.07585151,  0.22910478,  0.09092859],
       [ 0.38083699, -0.13130707,  0.05414608],
       [-0.16786925,  0.42898807,  0.09119147],
       [ 0.26288416,  0.02284962, -0.08650412],
       [ 0.40556906, -0.23529409, -0.15539194],
       [ 0.25828938,  0.08305889,  0.05971669],
       [-0.01333983,  0.3288417 ,  0.03704379],
       [-0.13133403,  0.44343312, -0.51745538]])

In [26]:
# Compute scores
rotated_scores = pd.DataFrame(places.dot(rotated_weight_matrix))
rotated_scores

,0,1,2
0,-1.144652,0.660634,-1.134468
1,0.340830,0.137161,1.135413
2,-1.197845,-0.539361,-0.202436
3,1.689490,-0.706512,-0.890694
4,0.870696,0.953851,0.565216
...,...,...,...
324,1.078728,-1.295060,0.166145
325,-1.209469,0.405216,0.343485
326,-0.100377,-1.571604,0.167956
327,0.380850,-1.078529,1.086948


## 13: Issues with SoVI's implementation of PCA

My main concern with using PCA to calculate a social vulnerability index is that PCA is an unsupervised learning technique that does not have any inherent relationship with vulnerability.
Rather than relying on knowledge about vulnerability or value judgements, SoVI uses PCA to determine components that explain the most variability in the data.
This is what Hinkel refers to as a "non-substantial argument", because nothing about the PCA process actually ties the components to the concept of vulnerability (Hinkel, 2011).

Additionally, in Cutter et al.'s original SoVI methodology, the authors interpret the meaning of each component, and reverse their directionality if needed (so higher values of the component are associated with a higher degree of vulnerability) before combining the components into a final index (Cutter et al., 2003).
For components that should theoretically both increase and decrease vulnerability, Cutter et al. took the absolute value (what do they mean by that???). Spielman et al. choose not to assign meanings to each individual component, because components are not actually trained to represent constructs like personal wealth and density of the built environment (Spielman et al., 2020).
A component may load highly on variables associated with these ideas, but these loadings are generated in an unsupervised manner, creating loadings to explain variance rather than to explain a concept.

Spielman et al. claim that they address the issue of directionality by switching the direction of variabiles before input into PCA, so that all variables are directly related with their theoretical contribution to vulnerability.
However, Schmidtlein et al. claim that "variables whose signs were adjusted prior to inclusion in the PCA may still load in a manner that indicates that the component would decrease vulnerability" (Schmidtlein et al., 2008).
I don't know how Python and other software calculate eigenvalues and eigenvectors, but I do know that my eigenvectors had the opposite sign of the Penn State tutorial's.
Mathematically, the sign of an eigenvector is arbitrary -- regardless of sign, the vector will obey the same relationship that defines eigenvectors.
If different software may produce eigenvectors in opposite directions, that raises concerns for me about automating SoVI... perhaps SoVI requires some manual interpretation?
But if SoVI requires manual interpretation, how valid can we really say that SoVI is across time?
And if SoVI is not comparable over time, does that make it less useful? 

One final potentiall issue with SoVI is that it treats all factors equally, summing all of the principal components (with eigenvalue greater than 1) with equal weight even though some components explain more variation in the data than others.
Would results be more stable if factors were weighted by the percent variance explained?

## 14: References
- Cutter, S. L., Boruff, B. J., & Shirley, W. L. (2003). Social Vulnerability to Environmental Hazards *: Social Vulnerability to Environmental Hazards. Social Science Quarterly, 84(2), 242–261. https://doi.org/10.1111/1540-6237.8402002
- Hinkel, J. (2011). “Indicators of vulnerability and adaptive capacity”: Towards a clarification of the science–policy interface. Global Environmental Change, 21(1), 198–208. https://doi.org/10.1016/j.gloenvcha.2010.08.002
- Schmidtlein, M. C., Deutsch, R. C., Piegorsch, W. W., & Cutter, S. L. (2008). A Sensitivity Analysis of the Social Vulnerability Index. Risk Analysis, 28(4), 1099–1114. https://doi.org/10.1111/j.1539-6924.2008.01072.x
- Spielman, S. E., Tuccillo, J., Folch, D. C., Schweikert, A., Davies, R., Wood, N., & Tate, E. (2020). Evaluating Social Vulnerability Indicators: Criteria and their Application to the Social Vulnerability Index. Natural Hazards, 100(1), 417–436. https://doi.org/10.1007/s11069-019-03820-z

## 15: Further reading
- https://stats.oarc.ucla.edu/spss/seminars/efa-spss/
- https://towardsdatascience.com/a-one-stop-shop-for-principal-component-analysis-5582fb7e0a9c
- https://builtin.com/data-science/step-step-explanation-principal-component-analysis
- https://towardsdatascience.com/x%E1%B5%80x-covariance-correlation-and-cosine-matrices-d2230997fb7
- https://towardsdatascience.com/principal-component-analysis-ac90b73f68f5
- https://vitalflux.com/pca-explained-variance-concept-python-example/
- https://stats.stackexchange.com/questions/151653/what-is-the-intuitive-reason-behind-doing-rotations-in-factor-analysis-pca-how
- https://stats.stackexchange.com/questions/143905/loadings-vs-eigenvectors-in-pca-when-to-use-one-or-another
- https://towardsdatascience.com/how-exactly-does-pca-work-5c342c3077fe ( I haven't been able to read this because I don't have a subscription )
- https://stats.stackexchange.com/questions/126885/methods-to-compute-factor-scores-and-what-is-the-score-coefficient-matrix-in

## 16: Validating Spielman et al.'s PCA function
Here, we will calculate PCA with varimax rotation using Spielman et al.'s function to verify their results.

In [27]:
# Import modules required for Spielman et al.'s code
# from scipy.stats.mstats import zscore as ZSCORE
import mdp as MDP
import copy
from operator import itemgetter

In [28]:
# Import data
places = pd.read_csv("places.csv")

# Drop id column
places = places.drop(axis = 1, columns = {"id"})

# Standardize data using sample standard deviation rather than population standard deviation in order to get numbers to match
for col in places.columns:
    places[col] = np.log10(places[col]) # the tutorial took a log transform to deal with skew
    places[col] = ( places[col] - np.mean(places[col]) ) / np.std(places[col], ddof=1) # ddof = 1 is to compute sample standard deviation instead of population standard deviation

In [29]:
# ONLY CHANGE HERE: I calculated the z-score beforehand in order to use the sample rather than population standard deviation to match the tutorial and previous results.
class SPSS_PCA:
	'''
	A class that integrates most (all?) of the assumptions SPSS imbeds in their
    implimnetation of principal components analysis (PCA), which can be found in
    thier GUI under Analyze > Dimension Reduction > Factor. This class is not
	intended to be a full blown recreation of the SPSS Factor Analysis GUI, but
	it does replicate (possibly) the most common use cases. Note that this class
	will not produce exactly the same results as SPSS, probably due to differences
	in how eigenvectors/eigenvalues and/or singular values are computed. However,
	this class does seem to get all the signs to match, which is not really necessary
	but kinda nice. Most of the approach came from the official SPSS documentation.

	References
	----------
	ftp://public.dhe.ibm.com/software/analytics/spss/documentation/statistics/20.0/en/client/Manuals/IBM_SPSS_Statistics_Algorithms.pdf
	http://spssx-discussion.1045642.n5.nabble.com/Interpretation-of-PCA-td1074350.html
	http://mdp-toolkit.sourceforge.net/api/mdp.nodes.WhiteningNode-class.html
	https://github.com/mdp-toolkit/mdp-toolkit/blob/master/mdp/nodes/pca_nodes.py

	Parameters
	----------
	inputs:  numpy array
			 n x k numpy array; n observations and k variables on each observation
	reduce:  boolean (default=False)
			 If True, then use eigenvalues to determine which factors to keep; all
			 results will be based on just these factors. If False use all factors.
	min_eig: float (default=1.0)
			 If reduce=True, then keep all factors with an eigenvalue greater than
			 min_eig. SPSS default is 1.0. If reduce=False, then min_eig is ignored.
	varimax: boolean (default=False)
			 If True, then apply a varimax rotation to the results. If False, then
			 return the unrotated results only.

	Attributes
	----------
	z_inputs:	numpy array
				z-scores of the input array.
	comp_mat:	numpy array
				Component matrix (a.k.a, "loadings").
	scores:		numpy array
				New uncorrelated vectors associated with each observation.
	eigenvals_all:	numpy array
				Eigenvalues associated with each factor.
	eigenvals:	numpy array
				Subset of eigenvalues_all reflecting only those that meet the
				criterion defined by parameters reduce and min_eig.
	weights:    numpy array
				Values applied to the input data (after z-scores) to get the PCA
				scores. "Component score coefficient matrix" in SPSS or
				"projection matrix" in the MDP library.
	comms: 		numpy array
				Communalities
	sum_sq_load: numpy array
				 Sum of squared loadings.
	comp_mat_rot: numpy array or None
				  Component matrix after rotation. Ordered from highest to lowest
				  variance explained based on sum_sq_load_rot. None if varimax=False.
	scores_rot:	numpy array or None
				Uncorrelated vectors associated with each observation, after
				rotation. None if varimax=False.
	weights_rot: numpy array or None
				Rotated values applied to the input data (after z-scores) to get
				the PCA	scores. None if varimax=False.
	sum_sq_load_rot: numpy array or None
				 Sum of squared loadings for rotated results. None if
				 varimax=False.

	'''

	def __init__(self, inputs, reduce=False, min_eig=1.0, varimax=False):
        # Step S1
		z_inputs = inputs  # seems necessary for SPSS "correlation matrix" setting (their default)
        
        # The rest is step S2
		# run base SPSS-style PCA to get all eigenvalues
		pca_node = MDP.nodes.WhiteningNode()  # settings for the PCA
		scores = pca_node.execute(z_inputs)  # base run PCA
		eigenvalues_all = pca_node.d   # rename PCA results

		# run SPSS-style PCA based on user settings
		pca_node = MDP.nodes.WhiteningNode(reduce=reduce, var_abs=min_eig)  # settings for the PCA
		scores = pca_node.execute(z_inputs)  # run PCA  (these have mean=0, std_dev=1)
		weights = pca_node.v  # rename PCA results (these might be a transformation of the eigenvectors)
		eigenvalues = pca_node.d   # rename PCA results
		component_matrix = weights * eigenvalues  # compute the loadings
		component_matrix = self._reflect(component_matrix)   # get signs to match SPSS
		communalities = (component_matrix**2).sum(1)   # compute the communalities
		sum_sq_loadings = (component_matrix**2).sum(0) # note that this is the same as eigenvalues
		weights_reflected = component_matrix/eigenvalues  # get signs to match SPSS
		scores_reflected = np.dot(z_inputs, weights_reflected)  # note that abs(scores)=abs(scores_reflected)

		if varimax:
			# SPSS-style varimax rotation prep
			c_normalizer = 1. / MDP.numx.sqrt(communalities)  # used to normalize inputs to varimax
			c_normalizer.shape = (component_matrix.shape[0],1)  # reshape to vectorize normalization
			cm_normalized = c_normalizer * component_matrix  # normalize component matrix for varimax

			# varimax rotation
			cm_normalized_varimax = self._varimax(cm_normalized)  # run varimax
			c_normalizer2 = MDP.numx.sqrt(communalities)  # used to denormalize varimax output
			c_normalizer2.shape = (component_matrix.shape[0],1)  # reshape to vectorize denormalization
			cm_varimax = c_normalizer2 * cm_normalized_varimax  # denormalize varimax output

			# reorder varimax component matrix
			sorter = (cm_varimax**2).sum(0)  # base the ordering on sum of squared loadings
			sorter = zip(sorter.tolist(), range(sorter.shape[0]))  # add index to denote current order
			sorter = sorted(sorter, key=itemgetter(0), reverse=True)  # sort from largest to smallest
			sum_sq_loadings_varimax, reorderer = zip(*sorter)  # unzip the sorted list
			sum_sq_loadings_varimax = np.array(sum_sq_loadings_varimax)  # convert to array
			cm_varimax = cm_varimax[:,reorderer]  # reorder component matrix

			# varimax scores
			cm_varimax_reflected = self._reflect(cm_varimax)  # get signs to match SPSS
			varimax_weights = np.dot(cm_varimax_reflected,
							  np.linalg.inv(np.dot(cm_varimax_reflected.T,
							  cm_varimax_reflected))) # CM(CM'CM)^-1
			scores_varimax = np.dot(z_inputs, varimax_weights)
		else:
			comp_mat_rot = None
			scores_rot = None
			weights_rot = None

		# assign output variables
		self.z_inputs = z_inputs
		self.scores = scores_reflected
		self.comp_mat = component_matrix
		self.eigenvals_all = eigenvalues_all
		self.eigenvals = eigenvalues
		self.weights = weights_reflected
		self.comms = communalities
		self.sum_sq_load = sum_sq_loadings
		self.comp_mat_rot = cm_varimax_reflected
		self.scores_rot = scores_varimax # PCA scores output
		self.weights_rot = varimax_weights # PCA weights output
		self.sum_sq_load_rot = sum_sq_loadings_varimax

	def _reflect(self, cm):
		# reflect factors with negative sums; SPSS default
		cm = copy.deepcopy(cm)
		reflector = cm.sum(0)
		for column, measure in enumerate(reflector):
			if measure < 0:
				cm[:,column] = -cm[:,column]
		return cm

	def _varimax(self, Phi, gamma = 1.0, q = 100, tol = 1e-6):
		# downloaded from http://en.wikipedia.org/wiki/Talk%3aVarimax_rotation
		# also here http://stackoverflow.com/questions/17628589/perform-varimax-rotation-in-python-using-numpy
		p,k = Phi.shape
		R = np.eye(k)
		d=0
		for i in range(q):
			d_old = d
			Lambda = np.dot(Phi, R)
			u,s,vh = np.linalg.svd(np.dot(Phi.T,np.asarray(Lambda)**3 - (gamma/p) *
							np.dot(Lambda, np.diag(np.diag(np.dot(Lambda.T,Lambda))))))
			R = np.dot(u,vh)
			d = np.sum(s)
			if d_old!=0 and d/d_old < 1 + tol:
				break
		return np.dot(Phi, R)

In [30]:
# Run Spielman et al.'s code
pca = SPSS_PCA(places.values, reduce=True, varimax=True)

In [31]:
# Test equality of (unrotated) component weight matrices
component_weight_matrix[[0,1,2]].round(5).eq( pd.DataFrame(pca.weights.round(5), index = ["climate", "housing", "health", "crime", "trans", "educate", "arts", "recreate", "economy"]) )

,0,1,2
climate,True,True,True
housing,True,True,True
health,True,True,True
crime,True,True,True
trans,True,True,True
educate,True,True,True
arts,True,True,True
recreate,True,True,True
economy,True,True,True


In [32]:
# Test equality of (unrotated) scores
pd.DataFrame(pca.scores.round(5)).eq(scores.round(5)).sum() # Note: there are 329 observations

0    329
1    329
2    329
dtype: int64

In [33]:
# Test equality of rotated component weight matrices
pd.DataFrame(pca.weights_rot)-rotated_weight_matrix
# the rounding method didn't work too well, but this illustrates that the rotated weights are quite similar

,0,1,2
0,-0.000245,0.001555,-0.000254
1,-0.000108,0.000220,-0.000465
2,0.000033,0.000254,0.000383
3,-0.000181,0.000131,-0.000956
4,0.000015,-0.000088,0.000024
5,0.000128,-0.000181,0.000611
6,-0.000047,0.000221,-0.000105
7,-0.000130,0.000073,-0.000701
8,-0.000019,-0.001146,-0.000975


In [34]:
# Test equality of rotated scores
diff = np.abs(pd.DataFrame(pca.scores_rot)-(rotated_scores))
max(diff.max()) # here we find a relatively minor maximum difference between the scores

0.010060633861689983